In [21]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
import time
import os
import urllib.request
from bs4 import BeautifulSoup
import re
import openpyxl as op
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from PIL import Image as pi


urls ='https://kream.co.kr/search?keyword=snow%20peak'

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(urls)

# action = ActionChains(driver) - 필요 없을 듯?

def imageLoad(path1, path2):
    wb = op.load_workbook(path1)
    ws = wb.active
 
    row_max = ws.max_row #최대행값 저장
    i=2 #2행부터 시작
    for r in range(2, row_max+1):
        file_name = ws.cell(row=i, column=1).value
        #Image 클래스의 객체 img 선언 : Image 클래스 선언시 매개변수는 이미지 파일 경로이다.
        img = Image(path2 + "/" + file_name)
        ws.add_image(img, "B"+str(i))

        #행마다 행크기 변경
        ws.row_dimensions[i].height = 111
        i=i+1 #1행 아래로 이동

    #Column 크기는 1번만 실행해도 됨(엑셀의 열을 생각해보자.)    
    ws.column_dimensions['B'].width = 18
    wb.save("img_result.xlsx")

def resizeImg(size, img_path, img_name):
    #이미지 객체 생성
    img = pi.open(img_path+"/"+img_name)
    #이미지 resize
    resize_img = img.resize(size)
    #수정한 이미지 저장
    resize_img.save(img_path+"/"+img_name, "png", quality=95)

def infinite_loop():
    # 최초 페이지 스크롤 설정
    # 스크롤 시키지 않았을 때의 전체 높이
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        # 윈도우 창을 0에서 위에서 설정한 전체 높이로 이동
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.0)
        # 스크롤 다운한 만큼의 높이를 신규 높이로 설정 
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        # 직전 페이지 높이와 신규 페이지 높이 비교
        if new_page_height == last_page_height:
            time.sleep(1.0)
            # 신규 페이지 높이가 이전과 동일하면, while문 break
            if new_page_height == driver.execute_script("return document.documentElement.scrollHeight"):
                break
        else:
            last_page_height = new_page_height

# infinite_loop() # 스크롤 내리기
            
xx = driver.find_elements_by_css_selector('div.search_result_item > a')

pro_link = []
for i in xx:
    href = i.get_attribute('href')
    pro_link.append(href)

wb = Workbook()
ws1 = wb.active
#sheet이름
ws1.title = "크롤링"
#제목 행 추가 
ws1.append(["파일명","이미지","상품 이름", "상품 부제목", "사이즈","구매 가격", "판매 가격","증감","발매가","URL"])
count = 1
for j in pro_link:
    driver.get(j)
    
    uni_num = re.sub(r'[^0-9]', '', j) # url 의 고유 번호 기준으로 사진을 저장하기 위함

    driver.find_element_by_css_selector(".slide_item").screenshot(str(uni_num)+'.png')
    resizeImg((150,150),r"C:\Users\Wook",uni_num + ".png")
    
    pro_titles = driver.find_element_by_css_selector("p.title:nth-child(2)")
    sub_titles = driver.find_element_by_css_selector(".sub_title")
    size = driver.find_element_by_css_selector("span.btn_text:nth-child(1)")
    buy_price = driver.find_element_by_css_selector("div.division_btn_box:nth-child(1) > a:nth-child(1) > div:nth-child(2) > span:nth-child(1) > em:nth-child(1)")
    sell_price = driver.find_element_by_css_selector("div.division_btn_box:nth-child(1) > a:nth-child(2) > div:nth-child(2) > span:nth-child(1) > em:nth-child(1)")
    flu_price = driver.find_element_by_css_selector(".fluctuation > p:nth-child(1)")
    rel_price = driver.find_element_by_css_selector("div.detail_box:nth-child(4) > dd:nth-child(2)")
    
    
    print(pro_titles.text,sub_titles.text,size.text ,buy_price.text, sell_price.text, flu_price.text , rel_price.text )
     
#행 추가 
    ws1.append([uni_num + ".png","",pro_titles.text,sub_titles.text,size.text ,buy_price.text, sell_price.text, flu_price.text,rel_price.text,j])
    print("{} / {}".format(count,len(pro_link)))
    count += 1
wb.save(filename='크롤링.xlsx')

imageLoad(r"C:\Users\Wook\크롤링.xlsx",r"C:\Users\Wook")

driver.quit()



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\Wook\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/2367208332.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/2367208332.py:74: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  xx = driver.find_elements_by_css_selector('div.search_result_item > a')
C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/2367208332.py:93: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(".slide_item").screensh

Snow Peak Giga Power Stove (Korean Ver.) 스노우 피크 강염 스토브 (국내 정식 발매 제품) ONE SIZE 517,000 506,000 5,000원 (-1.0%) 497,000원
1 / 40
Snow Peak Takibi My Table 스노우 피크 타키비 마이 테이블 ONE SIZE 270,000 250,000 0원 (0%) 198,000원
2 / 40
Snow Peak Flat Burner (Korean Ver.) 스노우 피크 플랫 버너 (국내 정식 발매 제품) ONE SIZE 162,000 153,000 0원 (0%) 138,000원
3 / 40
Snow Peak Titanium Single Mug 600(R) 스노우 피크 티타늄 싱글 머그 600(R) ONE SIZE 54,000 37,000 0원 (0%) 32,000원
4 / 40
Snow Peak Titanium Sierra Cup 스노우 피크 티탄 시에라 컵 ONE SIZE 32,000 - 2,000원 (+6.7%) 25,000원
5 / 40
Snow Peak Wire Tray Deep 1 Unit 스노우 피크 메쉬 트레이 1 유닛 딥 ONE SIZE 73,000 67,000 1,000원 (-1.4%) 64,000원
6 / 40
Snow Peak Tableware Set L 스노우 피크 식기 세트 L ONE SIZE 53,000 - 0원 (0%) 50,000원
7 / 40
Snow Peak Tableware Set L Family 스노우 피크 식기 세트 L 패밀리 ONE SIZE 187,000 135,000 12,000원 (-6.0%) 188,000원
8 / 40
Snow Peak Jikaro table 스노우 피크 지카로 테이블 ONE SIZE 420,000 380,000 35,000원 (-8.3%) 408,000원
9 / 40
Snow Peak Titanium Single Mug 450 스노우 피크 티타늄 싱글 머그 450 ONE SIZE 39,000 30,000

In [12]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
import time
import os
import urllib.request
from bs4 import BeautifulSoup
import re
import openpyxl as op
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from PIL import Image as pi


def imageLoad(path1, path2):
    wb = op.load_workbook(path1)
    ws = wb.active
 
    row_max = ws.max_row #최대행값 저장
    i=2 #2행부터 시작
    for r in range(2, row_max+1):
        file_name = ws.cell(row=i, column=1).value
        #Image 클래스의 객체 img 선언 : Image 클래스 선언시 매개변수는 이미지 파일 경로이다.
        img = Image(path2 + "/" + file_name)
        ws.add_image(img, "B"+str(i))

        #행마다 행크기 변경
        ws.row_dimensions[i].height = 111
        i=i+1 #1행 아래로 이동

    #Column 크기는 1번만 실행해도 됨(엑셀의 열을 생각해보자.)    
    ws.column_dimensions['B'].width = 18
    wb.save("img_result.xlsx")

def resizeImg(size, img_path, img_name):
    #이미지 객체 생성
    img = pi.open(img_path+"/"+img_name)
    #이미지 resize
    resize_img = img.resize(size)
    #수정한 이미지 저장
    resize_img.save(img_path+"/"+img_name, "png", quality=95)


li = ["https://kream.co.kr/products/56322","https://kream.co.kr/products/60765"]
wb = Workbook()

ws1 = wb.active
#sheet이름
ws1.title = "크롤링"
#제목 행 추가 
ws1.append(["파일명","이미지","상품 이름", "상품 부제목", "사이즈","구매 가격", "판매 가격","증감","발매가"])
count = 1
driver = webdriver.Chrome(ChromeDriverManager().install())
for urls in li:

    
    
    driver.get(urls)
    uni_num = re.sub(r'[^0-9]', '', urls) # url 의 고유 번호 기준으로 사진을 저장하기 위함
    print(uni_num)
    driver.find_element_by_css_selector(".slide_item").screenshot(str(uni_num)+'.png')
    resizeImg((150,150),r"C:\Users\Wook",uni_num + ".png")
    
    pro_titles = driver.find_element_by_css_selector("p.title:nth-child(2)")
    sub_titles = driver.find_element_by_css_selector(".sub_title")
    size = driver.find_element_by_css_selector("span.btn_text:nth-child(1)")
    buy_price = driver.find_element_by_css_selector("div.division_btn_box:nth-child(1) > a:nth-child(1) > div:nth-child(2) > span:nth-child(1) > em:nth-child(1)")
    sell_price = driver.find_element_by_css_selector("div.division_btn_box:nth-child(1) > a:nth-child(2) > div:nth-child(2) > span:nth-child(1) > em:nth-child(1)")
    flu_price = driver.find_element_by_css_selector(".fluctuation > p:nth-child(1)")
    rel_price = driver.find_element_by_css_selector("div.detail_box:nth-child(4) > dd:nth-child(2)")
    if size.text == "모든 사이즈":
        driver.find_element_by_css_selector("span.btn_text:nth-child(1)").click()
        pri = driver.find_elements_by_css_selector(".select_list > li > button > div > span")
        cnt = 1
        for ii in pri:
            print(ii.text,cnt,len(pri))
            cnt +=1
            
        
    print(pro_titles.text,sub_titles.text,size.text ,buy_price.text, sell_price.text, flu_price.text , rel_price.text )
     
#행 추가 
    ws1.append([uni_num + ".png","",pro_titles.text,sub_titles.text,size.text ,buy_price.text, sell_price.text, flu_price.text,rel_price.text])
    print("{} / {}".format(count,len(li)))
    count += 1
# set column width
# for col in ws1.columns:
#     max_length = 0
#     column = col[0].column

#     for cell in col:
#         if len(str(cell.value)) > max_length:
#             max_length = len(str(cell.value))

#     adjusted_width = (max_length + 2) * 1.2

#파일이름
wb.save(filename='크롤링.xlsx')

imageLoad(r"C:\Users\Wook\크롤링.xlsx",r"C:\Users\Wook")

# for i in pro_titles:
#     title = i.text
#     print(title)

# 추가해야할 것 -- 1.이미지 삭제
#                  2.로그인
#                  3.그래프 이미지 삽입
#                  4.여러 사이즈일 경우 어떻게 삽입을 할까?



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\Wook\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/1369646866.py:55: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


56322


C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/1369646866.py:63: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(".slide_item").screenshot(str(uni_num)+'.png')
C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/1369646866.py:66: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  pro_titles = driver.find_element_by_css_selector("p.title:nth-child(2)")
C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/1369646866.py:67: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  sub_titles = driver.find_element_by_css_selector(".sub_title")
C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/1369646866.py:68: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_S

모든 사이즈 1 30
199,000 2 30
250 3 30
208,000 4 30
255 5 30
200,000 6 30
260 7 30
207,000 8 30
265 9 30
199,000 10 30
270 11 30
215,000 12 30
275 13 30
205,000 14 30
280 15 30
230,000 16 30
285 17 30
220,000 18 30
290 19 30
250,000 20 30
295 21 30
구매입찰 22 30
300 23 30
300,000 24 30
305 25 30
구매입찰 26 30
310 27 30
구매입찰 28 30
320 29 30
구매입찰 30 30
Jordan 1 Low Cardinal Red 조던 1 로우 카디널 레드 모든 사이즈 199,000 190,000 44,000원 (+22.6%) 129,000원
1 / 2
60765
모든 사이즈 1 30
172,000 2 30
250 3 30
211,000 4 30
255 5 30
200,000 6 30
260 7 30
188,000 8 30
265 9 30
190,000 10 30
270 11 30
189,000 12 30
275 13 30
189,000 14 30
280 15 30
187,000 16 30
285 17 30
192,000 18 30
290 19 30
195,000 20 30
295 21 30
200,000 22 30
300 23 30
172,000 24 30
305 25 30
229,000 26 30
310 27 30
240,000 28 30
320 29 30
구매입찰 30 30
Jordan 1 Low Light Smoke Grey Black 조던 1 로우 라이트 스모크 그레이 블랙 모든 사이즈 172,000 197,000 1,000원 (+0.5%) 129,000원
2 / 2


In [34]:
pip install openpyxl 

Note: you may need to restart the kernel to use updated packages.


In [35]:
from openpyxl import Workbook

wb = Workbook()
ws1 = wb.active
#sheet이름
ws1.title = "네이버 기사 스크래핑"
#행 추가 
ws1.append(["제목", "링크", "신문사"])
#행 추가 
ws1.append(['1','2','3'])

#파일이름
wb.save(filename='네이버 기사 스크래핑.xlsx')

In [51]:
import re

string = 'https://kream.co.kr/products/57012'
numbers = re.sub(r'[^0-9]', '', string)
print(numbers)

57012


In [3]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
import time
import os
import urllib.request
from bs4 import BeautifulSoup
import re
import openpyxl as op
from openpyxl.drawing.image import Image
from PIL import Image as pi

urls ='https://kream.co.kr/login'

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(urls)

driver.find_element_by_css_selector("button.btn:nth-child(1)").click()
driver.find_element_by_css_selector("#log\.login").click()




====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\Wook\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/9522056.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/9522056.py:20: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector("button.btn:nth-child(1)").click()
C:\Users\Wook\AppData\Local\Temp/ipykernel_45292/9522056.py:21: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector("#log\.login").click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#log\.login"}
  (Session info: chrome=102.0.5005.61)
Stacktrace:
Backtrace:
	Ordinal0 [0x0067D953+2414931]
	Ordinal0 [0x0060F5E1+1963489]
	Ordinal0 [0x004FC6B8+837304]
	Ordinal0 [0x00529500+1021184]
	Ordinal0 [0x0052979B+1021851]
	Ordinal0 [0x00556502+1205506]
	Ordinal0 [0x005444E4+1131748]
	Ordinal0 [0x00554812+1198098]
	Ordinal0 [0x005442B6+1131190]
	Ordinal0 [0x0051E860+976992]
	Ordinal0 [0x0051F756+980822]
	GetHandleVerifier [0x008ECC62+2510274]
	GetHandleVerifier [0x008DF760+2455744]
	GetHandleVerifier [0x0070EABA+551962]
	GetHandleVerifier [0x0070D916+547446]
	Ordinal0 [0x00615F3B+1990459]
	Ordinal0 [0x0061A898+2009240]
	Ordinal0 [0x0061A985+2009477]
	Ordinal0 [0x00623AD1+2046673]
	BaseThreadInitThunk [0x76E26739+25]
	RtlGetFullPathName_UEx [0x77A88FEF+1215]
	RtlGetFullPathName_UEx [0x77A88FBD+1165]
